# Distributed Multi-Node, Multi-GPU Audio Transcription in ML Container Runtime

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import torch
# We can also use Snowpark for our analyses!
from typing import Dict
from pathlib import Path
import numpy as np
import shutil
from snowflake.snowpark.context import get_active_session
from snowflake.ml.ray.datasource import SFStageBinaryFileDataSource
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from snowflake.ml.runtime_cluster import scale_cluster, get_nodes
from snowflake.ml.ray.datasink import SnowflakeTableDatasink
import ray
import subprocess
import logging
session = get_active_session()

In [ ]:
create or replace stage AUDIO_FILES_STAGE ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE') DIRECTORY = (ENABLE = TRUE);

# UPLOAD ALL THE VIDEO FILES TO ABOVE STAGE

In [ ]:
import os
import glob
import subprocess

# --- Local directories for downloads and converted files ---
download_dir = "/tmp/mp4_downloads"
mp3_dir = "/tmp/mp3_outputs"
os.makedirs(download_dir, exist_ok=True)
os.makedirs(mp3_dir, exist_ok=True)

# --- List all MP4 files in the stage ---
file_list = session.sql(
    "LIST @MULTINODE_MULTIGPU_MYDB.AUDIO_TRANSCRIPTION_SCH.AUDIO_FILES_STAGE PATTERN='.*\\.mp4';"
).to_pandas()

print("Columns:", file_list.columns.tolist())
if not file_list.empty:
    print("Sample row:", file_list.iloc[0])

# --- Download MP4 files locally ---
for _, row in file_list.iterrows():
    stage_path = row.iloc[0]  # first column contains the stage path
    local_mp4 = os.path.join(download_dir, os.path.basename(stage_path))
    session.file.get(f"@{stage_path}", download_dir)  # no overwrite arg
    print(f"Downloaded {stage_path} to {local_mp4}")

# --- Convert each MP4 to MP3 ---
for mp4_file in glob.glob(os.path.join(download_dir, "*.mp4")):
    mp3_file = os.path.join(
        mp3_dir,
        os.path.splitext(os.path.basename(mp4_file))[0] + ".mp3"
    )
    subprocess.run([
        "ffmpeg", "-y", "-i", mp4_file, "-vn", "-acodec", "libmp3lame",
        "-ar", "44100", "-ac", "2", "-b:a", "192k", mp3_file
    ], check=True)
    print(f"Converted {mp4_file} to {mp3_file}")

# --- Upload MP3 files back to the stage ---
for mp3_file in glob.glob(os.path.join(mp3_dir, "*.mp3")):
    put_result = session.file.put(
        mp3_file,
        "@MULTINODE_MULTIGPU_MYDB.AUDIO_TRANSCRIPTION_SCH.AUDIO_FILES_STAGE",
        auto_compress=False,
        overwrite=True
    )
    print(f"Uploaded {mp3_file} to stage: {put_result}")


### Start with one node

In [ ]:
import ray

ray.init(ignore_reinit_error=True)

num_nodes = len([node for node in ray.nodes() if node["Alive"] == True])
print(num_nodes)


### Scale up to 5 nodes

In [ ]:
# Asynchronous scaling - function returns immediately after request is accepted
scale_cluster(5, is_async=True)

### Control ray logging

In [ ]:
def configure_ray_logger() -> None:
    #Configure Ray logging
    ray_logger = logging.getLogger("ray")
    ray_logger.setLevel(logging.CRITICAL)

    data_logger = logging.getLogger("ray.data")
    data_logger.setLevel(logging.CRITICAL)

    #Configure root logger
    logger = logging.getLogger()
    logger.setLevel(logging.CRITICAL)

    #Configure Ray's data context
    context = ray.data.DataContext.get_current()
    context.execution_options.verbose_progress = False
    context.enable_operator_progress_bars = False

configure_ray_logger()

In [ ]:
! ffmpeg -version

In [ ]:
print(int(ray.cluster_resources()['GPU']))

# See audio & video files in snowflake stage

In [ ]:
ls @AUDIO_FILES_STAGE

In [ ]:
from snowflake.ml.ray.datasource import SFStageBinaryFileDataSource

audio_source = SFStageBinaryFileDataSource(
    stage_location="@AUDIO_FILES_STAGE",
    database="MULTINODE_MULTIGPU_MYDB",
    schema="AUDIO_TRANSCRIPTION_SCH",
    file_pattern="*.mp3"
)

audio_dataset = ray.data.read_datasource(audio_source)


In [ ]:
audio_dataset.count()

### Get whisper model

In [ ]:
model_id = "openai/whisper-large-v3"
batch_size = 30
is_cuda_available = torch.cuda.is_available()
device = torch.device("cuda" if is_cuda_available else "cpu")
torch_dtype = torch.float16 if is_cuda_available else torch.float32
print(device)
print(torch_dtype)

### Distributed inferencing

In [ ]:
import pandas as pd
import tempfile
import os

class TranscribeAudioUpdated:
    def __init__(self):
        # initialize model here so that model can be put into correct GPU/node
        model = AutoModelForSpeechSeq2Seq.from_pretrained(
            model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
        )
        model.to(device)
        processor = AutoProcessor.from_pretrained(model_id)
        self.pipe = pipeline(
            "automatic-speech-recognition",
            model=model,
            tokenizer=processor.tokenizer,
            feature_extractor=processor.feature_extractor,
            max_new_tokens=128,
            chunk_length_s=30,
            batch_size=batch_size,
            return_timestamps=True,
            torch_dtype=torch_dtype,
            device=device,
            generate_kwargs={"language": "hindi"}
        )

    def __call__(self, batch: pd.DataFrame) -> pd.DataFrame:
        temp_files = []
        try:
            # Write each binary to a temporary file.
            for binary_content in batch["file_binary"]:
                # Use an appropriate suffix (e.g., .mp3) based on your audio format.
                tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
                tmp_file.write(binary_content)
                tmp_file.close()
                temp_files.append(tmp_file.name)
            
            # Use the temporary file paths for inference.
            predictions = self.pipe(temp_files)
            assert len(predictions) == len(batch)
            outputs = [str(generated_audio["text"]).strip() for generated_audio in predictions]
            batch['outputs'] = outputs
            batch.drop(columns=['file_binary'], inplace=True)
        finally:
            # Clean up temporary files.
            for file_path in temp_files:
                try:
                    os.remove(file_path)
                except OSError:
                    pass
        return batch

In [ ]:
transcribed_ds = audio_dataset.map_batches(TranscribeAudioUpdated,
        batch_size=batch_size,
        batch_format='pandas',
        concurrency=5,
        num_gpus=1,
)

In [ ]:
drop table if exists WHISPER_DEMO_OUTPUT

In [ ]:
datasink = SnowflakeTableDatasink(
    table_name="WHISPER_DEMO_OUTPUT",
    database=session.get_current_database(),
    schema=session.get_current_schema(),
    auto_create_table=True
)

In [ ]:
transcribed_ds.write_datasink(datasink)

In [ ]:
session.table("WHISPER_DEMO_OUTPUT").show()

In [ ]:
select * from WHISPER_DEMO_OUTPUT

In [ ]:
-- Create a new table to store Hindi and English translations
CREATE OR REPLACE TABLE WHISPER_DEMO_TRANSLATION AS
SELECT
    *,
    SNOWFLAKE.CORTEX.COMPLETE(
        'openai-gpt-4.1',
        CONCAT(
            'Translate the following Hindi conversation to English, preserving the meaning and context as a conversation:\n',
            outputs
        )
    ) AS english_translation
FROM WHISPER_DEMO_OUTPUT;

-- View the new table
SELECT * FROM WHISPER_DEMO_TRANSLATION;
